In [1]:
import pandas as pd

In [2]:
pd.__version__

'1.2.4'

## Assess SRA Search Results

### Load Results

In [3]:
# read in k=31 SRA search results
ms31 = pd.read_csv("../output.genome-magsearch/results/jb.k31.csv",
                   sep=",",
                   quotechar="'",
                   header=0,
                   names=["search_genome", "metagenome", "containment"])
ms31

,search_genome,metagenome,containment
0,641736108,/group/ctbrowngrp/irber/data/wort-data/wort-sr...,0.038136
1,MT027_1,/group/ctbrowngrp/irber/data/wort-data/wort-sr...,0.022670
2,2574179741,/group/ctbrowngrp/irber/data/wort-data/wort-sr...,0.011445
3,641736108,/group/ctbrowngrp/irber/data/wort-data/wort-sr...,0.012712
4,641736108,/group/ctbrowngrp/irber/data/wort-data/wort-sr...,0.012712
...,...,...,...
1449,2574179741,/group/ctbrowngrp/irber/data/wort-data/wort-sr...,0.014306
1450,2574179741,/group/ctbrowngrp/irber/data/wort-data/wort-sr...,0.014306
1451,641736108,/group/ctbrowngrp/irber/data/wort-data/wort-sr...,0.016949
1452,SSC5_1,/group/ctbrowngrp/irber/data/wort-data/wort-sr...,0.347382


In [4]:
# Fix names so it's easier to query
ms31['search_genome'] = ms31['search_genome'].str.replace(r"'(?P<id>.*)'", lambda m: m.group("id"), regex=True)
ms31['metagenome'] = ms31['metagenome'].str.replace(r".*/(?P<id>.*).sig.*", lambda m: m.group("id"), regex=True)
ms31['accession'] = ms31['search_genome'].str.split(" ", 1, expand=True)[0]
ms31

,search_genome,metagenome,containment,accession
0,641736108,ERR3500822,0.038136,641736108
1,MT027_1,ERR2856158,0.022670,MT027_1
2,2574179741,SRR13872429,0.011445,2574179741
3,641736108,ERR3500861,0.012712,641736108
4,641736108,SRR14000579,0.012712,641736108
...,...,...,...,...
1449,2574179741,SRR12529294,0.014306,2574179741
1450,2574179741,ERR2145013,0.014306,2574179741
1451,641736108,SRR6748051,0.016949,641736108
1452,SSC5_1,ERR1868849,0.347382,SSC5_1


In [5]:
# print some summary info for 50%+ containment
search_ksize = 31
num_search_genomes = len(ms31["search_genome"].unique())
num_search_genomes_with_matches = len(ms31[ms31['containment'] > 0.5]["search_genome"].unique())
num_unique_metagenome_matches = len(ms31[ms31['containment'] > 0.5]["metagenome"].unique())

print(f"Search ksize: {search_ksize}")
print(f"# Search genomes: {num_search_genomes}")
print(f"# Search genomes with 'good' metagenome matches: {num_search_genomes_with_matches}")
print(f"# Metagenomes with >50% containment of at least one search genome: {num_unique_metagenome_matches}")

Search ksize: 31
# Search genomes: 10
# Search genomes with 'good' metagenome matches: 7
# Metagenomes with >50% containment of at least one search genome: 97


In [6]:
# Summarize 30% containment, just out of curiosity
search_ksize = 31
c30_matches = len(ms31[ms31['containment'] > 0.3]["metagenome"].unique())
print(f"Search ksize: {search_ksize}")
print(f"# Metagenomes with >30% containment of at least one search genome: {c30_matches}")

Search ksize: 31
# Metagenomes with >30% containment of at least one search genome: 209


## Explore & Summarize >50% containment results

In [7]:
#subset to just >50% containment; sort by containment & look at top metagenome matches
ms31_c50 = ms31[ms31['containment'] > 0.5]
ms31_c50 = ms31_c50.sort_values(by="containment", ascending=False, ignore_index=True)
ms31_c50

,search_genome,metagenome,containment,accession
0,SSC5_1,SRR6706337,1.000000,SSC5_1
1,637000182,SRR4046667,0.997683,637000182
2,641736108,SRR13167625,0.991525,641736108
3,641736108,SRR11742806,0.988701,641736108
4,GML10_1,ERR2856163,0.982361,GML10_1
...,...,...,...,...
92,641736108,SRR6748011,0.519774,641736108
93,641736108,SRR13320442,0.514124,641736108
94,SSC5_1,ERR4765900,0.509579,SSC5_1
95,GML10_1,ERR2856187,0.503392,GML10_1


## Save some better formatted results

In [8]:
# save sorted containment-50 csvs
!mkdir -p ../output.genome-magsearch/processed_results

ms31_c50.to_csv("../output.genome-magsearch/processed_results/jb.sra-search.k31-c50.csv", index=False)

In [9]:
# save list of all unique matched metagenomes
m31_metagenomes = ms31_c50["metagenome"].unique()
!mkdir -p ../output.magsearch/processed_results
with open('../output.genome-magsearch/processed_results/ms31_c50.unique-metagenomes.txt', 'w') as out:
    for mg in list(m31_metagenomes):
        out.write(f"{mg}\n")
m31_metagenomes

array(['SRR6706337', 'SRR4046667', 'SRR13167625', 'SRR11742806',
       'ERR2856163', 'ERR3139978', 'SRR953870', 'SRR924899', 'SRR925109',
       'SRR925231', 'SRR953949', 'SRR925351', 'SRR924936', 'SRR925025',
       'SRR925035', 'SRR4046664', 'ERR2856162', 'ERR3139977',
       'SRR6706335', 'ERR4765908', 'ERR4765907', 'ERR4765912',
       'SRR2566039', 'ERR4407004', 'SRR6706336', 'ERR2856161',
       'ERR3139976', 'ERR4765909', 'ERR4765911', 'SRR6262267',
       'ERR4765910', 'ERR4765913', 'SRR13167624', 'ERR4765906',
       'SRR4046669', 'SRR13622618', 'ERR4407001', 'SRR5575603',
       'SRR13167600', 'SRR953975', 'SRR13167601', 'SRR924787',
       'SRR925040', 'SRR925090', 'SRR925353', 'SRR924872', 'SRR925270',
       'SRR924942', 'SRR953785', 'ERR4406976', 'SRR5405808', 'SRR3726351',
       'SRR060153', 'SRR13167511', 'SRR512770', 'SRR5535735',
       'SRR6748171', 'SRR4046665', 'ERR2856184', 'ERR2856183',
       'SRR6748170', 'ERR2856182', 'ERR3500848', 'SRR6748172',
       'SRR5